# Libraries

In [4]:
import pandas as pd
import numpy as np
import os
import sys

# Inputs

In [ ]:
EN_PATH = '../Data/Input/conceptnet_en_full.csv'

# Purpose
***

Allow an agent to interact with the full relation universe and create a full knowledge graph.

### Phase 1. 
***
Load the english triples

In [7]:
english_triples = pd.read_csv(EN_PATH)
english_triples.head()

,uri,relation,start,end,weight,dataset,sources,surfaceText,license,context
0,"/a/[/r/Antonym/,/c/en/0/n/,/c/en/1/]",/r/Antonym,/c/en/0/n,/c/en/1,"{""dataset"": ""/d/wiktionary/fr"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
1,"/a/[/r/Antonym/,/c/en/12_hour_clock/n/,/c/en/2...",/r/Antonym,/c/en/12_hour_clock/n,/c/en/24_hour_clock,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
2,"/a/[/r/Antonym/,/c/en/24_hour_clock/n/,/c/en/1...",/r/Antonym,/c/en/24_hour_clock/n,/c/en/12_hour_clock,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
3,"/a/[/r/Antonym/,/c/en/5/n/,/c/en/3/]",/r/Antonym,/c/en/5/n,/c/en/3,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
4,"/a/[/r/Antonym/,/c/en/a.c/n/,/c/en/d.c/]",/r/Antonym,/c/en/a.c/n,/c/en/d.c,"{""dataset"": ""/d/wiktionary/fr"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN


#### Phase 1.A 
***
Preprocess and prepare the concept data

In [ ]:
def clean_column(df, col):
    col_string = df[col].astype(str)
    col_string = col_string.str.split('/')
    col_string = col_string.str[-1]
    df[f'{col}_cleaned'] = col_string
    return df

def preprocess_data(df):
    df_copy = df.copy()
    for col in ['relation', 'start', 'end']:
        df_copy = clean_column(df_copy, col)
    return df_copy

cleaned_english_triples = preprocess_data(english_triples.head())
cleaned_english_triples

,uri,relation,start,end,weight,dataset,sources,surfaceText,license,context,relation_cleaned
0,"/a/[/r/Antonym/,/c/en/0/n/,/c/en/1/]",/r/Antonym,/c/en/0/n,/c/en/1,"{""dataset"": ""/d/wiktionary/fr"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN,Antonym
1,"/a/[/r/Antonym/,/c/en/12_hour_clock/n/,/c/en/2...",/r/Antonym,/c/en/12_hour_clock/n,/c/en/24_hour_clock,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN,Antonym
2,"/a/[/r/Antonym/,/c/en/24_hour_clock/n/,/c/en/1...",/r/Antonym,/c/en/24_hour_clock/n,/c/en/12_hour_clock,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN,Antonym
3,"/a/[/r/Antonym/,/c/en/5/n/,/c/en/3/]",/r/Antonym,/c/en/5/n,/c/en/3,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN,Antonym
4,"/a/[/r/Antonym/,/c/en/a.c/n/,/c/en/d.c/]",/r/Antonym,/c/en/a.c/n,/c/en/d.c,"{""dataset"": ""/d/wiktionary/fr"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN,Antonym


# Phase 2. Seed the agent with random english relations
***